In [7]:
import pandas as pd
import time
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense,Dropout


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

data_dir = 'drive/MyDrive/505/final/'

In [2]:
df = pd.read_csv('../dataset/wiki/shuffled_GPT_wiki_intro.csv', header=None)

/var/folders/h4/qv9ymlxn5v54y1vnffsnvmlr0000gn/T/ipykernel_801/2966890944.py:1: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../dataset/wiki/shuffled_GPT_wiki_intro.csv', header=None)


In [3]:
df = df[:200000]

In [10]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

# 文本预处理
max_words = 10000  # 仅使用最常见的10000个词
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(train_data.iloc[:, 1])  # 第三列是文本

X_train = tokenizer.texts_to_sequences(train_data.iloc[:, 1])
X_test = tokenizer.texts_to_sequences(test_data.iloc[:, 1])

max_sequence_length = 100 
X_train = pad_sequences(X_train, maxlen=max_sequence_length)
X_test = pad_sequences(X_test, maxlen=max_sequence_length)

# 标签预处理
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(train_data.iloc[:, 2])  # 第二列是标签
y_test = label_encoder.transform(test_data.iloc[:, 2])

# 建立GRU模型
embedding_dim = 100
hidden_units = 64

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(GRU(units=hidden_units))
model.add(Dropout(0.5)) 
model.add(Dense(1, activation='sigmoid'))

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [11]:
from keras.callbacks import EarlyStopping
training_start_time = time.time()

# 训练模型
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2,callbacks=[early_stopping])

# 记录训练结束时间
training_end_time = time.time()

# 记录推理开始时间
inference_start_time = time.time()
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)

# 记录推理结束时间
inference_end_time = time.time()


Epoch 1/5
4000/4000 [==============================] - 83s 21ms/step - loss: 0.2189 - accuracy: 0.9107 - val_loss: 0.1574 - val_accuracy: 0.9380
Epoch 2/5
4000/4000 [==============================] - 82s 20ms/step - loss: 0.1254 - accuracy: 0.9536 - val_loss: 0.1555 - val_accuracy: 0.9392
Epoch 3/5
4000/4000 [==============================] - 82s 20ms/step - loss: 0.0829 - accuracy: 0.9698 - val_loss: 0.1758 - val_accuracy: 0.9393
Epoch 4/5
4000/4000 [==============================] - 83s 21ms/step - loss: 0.0519 - accuracy: 0.9820 - val_loss: 0.2169 - val_accuracy: 0.9360
Epoch 5/5
1250/1250 [==============================] - 6s 5ms/step


In [12]:
model.save('gru.h5')  # creates a HDF5 file 'my_model.h5'

In [15]:
accuracy = accuracy_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)
auc = roc_auc_score(y_test, y_pred)

# 打印指标
print(f'Test Accuracy: {accuracy * 100:.2f}%')
print(f'Test F1 Score: {f1:.4f}')
print(f'Test AUC: {auc:.4f}')
print(f'Training Time: {training_end_time - training_start_time:.2f} seconds')
print(f'Inference Time: {inference_end_time - inference_start_time:.2f} seconds')

# 将指标保存到CSV文件
results_df = pd.DataFrame({
    'Metric': ['Accuracy', 'F1 Score', 'AUC', 'Training Time', 'Inference Time'],
    'Value': [accuracy, f1, auc, training_end_time - training_start_time, inference_end_time - inference_start_time]
})

results_df.to_csv('../dataset/gru_model_results.csv', index=False)

Test Accuracy: 93.92%
Test F1 Score: 0.9395
Test AUC: 0.9840
Training Time: 412.35 seconds
Inference Time: 6.65 seconds
